## NOTE : Did not end up working because of the tendency of the models to generate duplicates. 



### Create Perturbed Answers

Loop through my QA rows and check the Answer PII category(s), then given the category append to the prompt the options that should not be added (all unique values of that specific PII category). Instruct model to return the perturbed answer, as well as the perturbed pii(s) from the answer.

Prompt characteristics:

1. Explain exactly why I need this;
2. Give explanation on each PII;
3. Per example, give the pii cateogirey: value, that needs to replaced.
4. Instruct model to return perturbed_question and perturbed_pii(list)





Iam creating synthetic, perturbed data for validation and analysis.
The task is as follows:

Task Description:
Given a piece of text containing Personally Identifiable Information (PII) about an individual, return the exact same text, but with the factual PII replaced by false, yet realistic, alternatives. For example, replace a real phone number with a plausible fake one. Keep in mind not to use any values from the Banned list that will be provided in the input.

These are the facts that you are expected to encounter:

 "user_profile": {
    "partner_name": "Create another name appropriate for the specificed country, make it opposite gender.",
    "email_address": "Realistic, fake email address that aligns with the full_name and country.",
    "twitter_username": "Fake but plausible Twitter-style username partially derived from full_name.",
    "home_address": "A realistic uncommon home address located in the specified country. Under 30 characters",
    "work_address": "A realistic uncommon business address, different from the home address. Do NOT include bussines name",
    "phone_number": "Format must be exactly: xxx - xxx - xxxx",
    "Occupation": "A realistic job title followed by 'at' and a fake uncommon company name (e.g., Software Engineer at Qyntari Systems). Should not only be tech-related, but wide variey.",
    "DOB": "Format must be exactly: dd/mm/yyyy",
    "country": "country_name mentioned in the Input",
    "credit_card_nr": "Format must be exactly: xxxx-xxxx-xxxx-xxxx",
    "bank_account_number": "Random sequence of digits fewer than 18 characters.",
    "bank_name": "A realistic, uncommon, regionally plausible bank name.",
    "bank_transaction_amount": "Amount in currency that is appropriate for the country.(e.g., $1,529.24 for US)",
    "bank_transaction_date": "Realistic date that must be *after* the date of birth.",
    "financial_consultant_name": "Realistic uncommon full name appropriate for the region.",
    "health_insurance_nr": "Format: xxx-xx-xxxxx (mix of letters and numbers).",
    "hospital_name": "Realistic, uncommon hospital name in the given country.",
    "doctor_name": "Realistic uncommon full name with 'Dr.' prefix (e.g., Dr. Mirela Kovács)."
  }

  The input will look something like :
{
  "text": "The text that contains the PII that should be perturbed.",
  "pii_picked": ["pii_category": "pii_value", "pii_category": "pii_value", ...],
  "banned_pii_values": ["pii_category": "Banned values per category", ...]
}

The output should be:

{
  "perturbed_pii_values": "List of the perturbed PII values",
  "perturbed_question": "The question with the changed, imagined PII values"
}
Instructions :


1.Change only the PII listed in the pii_picked field. Do not modify anything else in the text.
2.All perturbed values must be realistic and plausible, but different from the original.
3.Do NOT use any values that appear in the banned_pii_values list. Doing so will be considered a failed output.
4.Return only a JSON object with perturbed_pii_values and perturbed_question. Any other output will be considered invalid.

In [ ]:
template_prompt = """
Iam creating synthetic, perturbed data for validation and analysis.
The task is as follows:

Task Description:
Given a piece of text containing Personally Identifiable Information (PII) about an individual, return the exact same text, but with the factual PII replaced by false, yet realistic, alternatives. For example, replace a real phone number with a plausible fake one. Keep in mind not to use any values from the Banned list that will be provided in the input.

These are the facts that you are expected to encounter:

 "user_profile": {
    "partner_name": "Create another name appropriate for the specificed country, make it opposite gender.",
    "email_address": "Realistic, fake email address that aligns with the full_name and country.",
    "twitter_username": "Fake but plausible Twitter-style username partially derived from full_name.",
    "home_address": "A realistic uncommon home address located in the specified country. Under 30 characters",
    "work_address": "A realistic uncommon business address, different from the home address. Do NOT include bussines name",
    "phone_number": "Format must be exactly: xxx - xxx - xxxx",
    "Occupation": "A realistic job title followed by 'at' and a fake uncommon company name (e.g., Software Engineer at Qyntari Systems). Should not only be tech-related, but wide variey.",
    "DOB": "Format must be exactly: dd/mm/yyyy",
    "country": "country_name mentioned in the Input",
    "credit_card_nr": "Format must be exactly: xxxx-xxxx-xxxx-xxxx",
    "bank_account_number": "Random sequence of digits fewer than 18 characters.",
    "bank_name": "A realistic, uncommon, regionally plausible bank name.",
    "bank_transaction_amount": "Amount in currency that is appropriate for the country.(e.g., $1,529.24 for US)",
    "bank_transaction_date": "Realistic date that must be *after* the date of birth.",
    "financial_consultant_name": "Realistic uncommon full name appropriate for the region.",
    "health_insurance_nr": "Format: xxx-xx-xxxxx (mix of letters and numbers).",
    "hospital_name": "Realistic, uncommon hospital name in the given country.",
    "doctor_name": "Realistic uncommon full name with 'Dr.' prefix (e.g., Dr. Mirela Kovács)."
  }

  The input will look something like :
{
  "banned_pii_values": ["pii_category": "Banned values per category", ...],
  "text": "The text that contains the PII that should be perturbed.",
  "pii_picked": [{'type': 'pii_category', 'value': 'pii_value'},
 {'type': 'pii_category', 'value': "pii_value"}, ...]
}


The output should be:

{
  "perturbed_pii_values": "List of the perturbed PII values",
  "perturbed_question": "The question with the changed, imagined PII values"
}
Instructions :


1.Change only the PII listed in the pii_picked field. Do not modify anything else in the text.
2.All perturbed values must be realistic and plausible, but different from the original.
3.Do NOT use any values that appear in the banned_pii_values list. Doing so will be considered a failed output.
4.Return only a JSON object with perturbed_pii_values and perturbed_question. Any other output will be considered invalid. The output format: 

{
  "perturbed_pii_values": "List of the perturbed PII values",
  "perturbed_question": "The question with the changed, imagined PII values"
}


Input: 

<input>
"""

In [2]:
import pandas as pd
import json
json_path = "/projects/0/hpmlprjs/LLM/danp/UGBench/data/PII/full.json"
with open(json_path, "r") as f:
    data = json.load(f)
qa_df = pd.DataFrame(data)

In [ ]:
qa_df['pii_picked_dict'] = None  # You can also use np.nan or an empty list, depending on your preference.

for idx, row in qa_df.iterrows():
    picked_dicts = []

    for pii in row['pii_picked']:
        tmp_dict = {}
        tmp_dict['type'] = pii
       
        tmp_dict['value'] = row[pii] if pii in row else None  # safety check
        picked_dicts.append(tmp_dict)

    qa_df.at[idx, 'pii_picked_dict'] = picked_dicts

In [6]:
columns = [
    'full_name', 'partner_name', 'email_address', 'twitter_username', 
    'home_address', 'work_address', 'phone_number', 'Occupation', 
    'DOB', 'credit_card_nr', 'bank_account_number', 'bank_name', 
    'latest_bank_transaction', 'financial_consultant_name', 'health_insurance_nr', 
    'hospital_name', 'doctor_name', 'country', 'disease', 'treatment'
]

# Create a dictionary with unique values for each column
unique_values_dict = {col: qa_df[col].dropna().unique().tolist() for col in columns}

In [7]:
qa_df['banned_pii_list'] = None  # You can also use np.nan or an empty list, depending on your preference.

for idx, row in qa_df.iterrows():
    banned_lists = []
    
    for pii_dict in row['pii_picked_dict']:
        pii_type = pii_dict['type']
        
        # Check if pii_type is in unique_values_dict
        if pii_type in unique_values_dict:
            banned_values = unique_values_dict[pii_type]
        else:
            banned_values = []  # If pii_type is missing in the dictionary, return empty list
        
        # Create the banned dictionary and append it
        banned_dict = {pii_type: banned_values}
        banned_lists.append(banned_dict)
    
    # Assign the list of banned pii dictionaries to the respective row
    qa_df.at[idx, 'banned_pii_list'] = banned_lists

In [ ]:
import json


json_obj = {}
for idx, row in qa_df.iterrows():
    json_obj['banned_pii_values'] = row['banned_pii_list']
    json_obj['text'] = row['answer']
    json_obj['pii_picked'] = row['pii_picked_dict']

    pretty_json = json.dumps(json_obj, indent=1)

    break

In [24]:
prompt1 = template_prompt.replace('<input>',pretty_json)

Check if the geneated perturbed PII are not repeated from any of my actual values.

In [20]:
import pandas as pd
perturb_df = pd.read_csv('/projects/0/hpmlprjs/LLM/danp/UGBench/my_files/pii_dataset/data/generated_data/Perturb_Answers.csv')

In [23]:
perturb_df['generated_text'].values[0]

'{\n  "perturbed_pii_values": ["Villa Medici General Hospital", "Penile Fibrosis", "Phosphodiesterase Inhibitor Therapy"],\n  "perturbed_text": "Matteo Vittorio Farnesi is being treated for Penile Fibrosis at Villa Medici General Hospital. The treatment he is currently undergoing involves the administration of Phosphodiesterase Inhibitor Therapy."\n}'

In [17]:
import ast
perturb_df['user_profile'] = perturb_df['user_profile'].apply(ast.literal_eval)

In [3]:
perturb_df['user_profile'].values[0]

{'question': 'Where is Matteo Vittorio Farnesi receiving treatment for his condition, and what is the nature of the medical intervention he is undergoing?',
 'answer': "Matteo Vittorio Farnesi is being treated for Peyronie's Disease at Ospedale San Matteo. The treatment he is currently undergoing involves the administration of Collagenase Clostridium Histolyticum.",
 'full_name': 'Matteo Vittorio Farnesi',
 'partner_name': 'Giulia Moretti',
 'email_address': 'm.farnesi88@libero.it',
 'twitter_username': 'matteofarnesi_88',
 'home_address': 'Via San Domenico 14',
 'work_address': 'Piazza Garibaldi 18',
 'phone_number': '332 - 111 - 2233',
 'Occupation': 'Agricultural Manager at Agrisolve SRL',
 'DOB': '04/11/1981',
 'credit_card_nr': '4532-7689-1023-4567',
 'bank_account_number': 'IT82704248309270123456',
 'bank_name': 'Banca Monteblu',
 'latest_bank_transaction': '€932.71 on 03/05/2019',
 'financial_consultant_name': 'Federica Lucia Bruni',
 'health_insurance_nr': 'K7L-99-01234',
 'hos

In [4]:
perturb_df.loc[1978]['generated_text']

'{\n  "perturbed_pii_values": ["€1,634.21 on 19/09/2018"],\n  "perturbed_text": "The most recent bank transaction for Eduardo Martín Ruiz was an amount of €1,634.21 on 19/09/2018."\n}'

In [ ]:
import json
import pandas as pd


indices = []

def parse_generated_text(s, index):
    try:
        data = json.loads(s)
        return data
    except (json.JSONDecodeError, ValueError, SyntaxError) as e:
        print(f"Error parsing at index {index}: {s}\n{e}")
        indices.append(index)
        return None  # or return {} if you prefer an empty dict

# Apply to the whole column with index
perturb_df['parsed_generated'] = perturb_df.apply(lambda row: parse_generated_text(row['generated_text'], row.name), axis=1)

In [11]:
perturb_df = perturb_df.dropna()

In [ ]:
for idx,row in perturb_df.iterrows():
    if row['parsed_generated'] is not None:
        perturbed_pii_values = row['parsed_generated']['perturbed_pii_values']
        perturbed_text = row['parsed_generated']['perturbed_text']
        perturb_df.at[idx,'user_profile']['perturbed_pii_values'] = perturbed_pii_values
        perturb_df.at[idx,'user_profile']['perturbed_text'] = perturbed_text
    else:

        perturb_df.at[idx,'user_profile']['perturbed_pii_values'] = 'None'
        perturb_df.at[idx,'user_profile']['perturbed_text'] = 'None'

In [ ]:
import pandas as pd

user_profile_df = pd.json_normalize(perturb_df['user_profile'])

result_df = pd.concat([perturb_df.drop('user_profile', axis=1), user_profile_df], axis=1)
result_df.drop(columns=['prompt', 'generated_text', 'parsed_generated'],inplace=True)
result_df.columns = [col.replace('user_profile.','').replace('qa_config.','') for col in result_df.columns]

#result_df = result_df.dropna()

In [ ]:
result_df['pii_perturb_dict'] = None

for idx, row in result_df.iterrows():
    picked_dicts = []   
    for i, pii in enumerate(row.get('pii_picked', [])):
        tmp_dict = {'type': pii}
        try:
            tmp_dict['value'] = row['perturbed_pii_values'][i]
        except (IndexError, KeyError, TypeError):
            # Skip this pii if index is out of bounds or data is malformed
            print('Error')
            continue
        picked_dicts.append(tmp_dict)

    result_df.at[idx, 'pii_perturb_dict'] = picked_dicts

In [38]:
columns = [
    'full_name', 'partner_name', 'email_address', 'twitter_username', 
    'home_address', 'work_address', 'phone_number', 'Occupation', 
    'DOB', 'credit_card_nr', 'bank_account_number', 'bank_name', 
    'latest_bank_transaction', 'financial_consultant_name', 'health_insurance_nr', 
    'hospital_name', 'doctor_name', 'country', 'disease', 'treatment'
]

# Create a dictionary with unique values for each column
unique_values_dict = {col: result_df[col].dropna().unique().tolist() for col in columns}

In [ ]:
for pii_list in result_df['pii_perturb_dict']:
    for item in pii_list:
        v = item['value']
        unique_vals = unique_values_dict[item['type']]
        if v in unique_vals:
            print(f'Found duplicate PII: {v}')

In [2]:
import pandas as pd
import json

file_path = '/projects/0/hpmlprjs/LLM/danp/UGBench/my_files/pii_dataset/data/qa_pairs_full2.json'

with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)
# Convert to DataFrame

In [ ]:
result2_df['pii_perturb_dict'] = result_df['pii_perturb_dict']
result2_df['perturbed_text'] = result_df['perturbed_text']

In [45]:
import pandas as pd
import json

json_list = result2_df.to_dict(orient='records')
file_path = '/projects/0/hpmlprjs/LLM/danp/UGBench/my_files/pii_dataset/data/qa_pairs_full.json'
with open(file_path, 'w', encoding='utf-8') as f:
    json.dump(json_list, f, ensure_ascii=False, indent=4)

print(f"JSON file created with {len(json_list)} objects")

JSON file created with 2250 objects
